In [2]:
pip install pandas tabulate PyMuPDF


Defaulting to user installation because normal site-packages is not writeable
  Using cached tabulate-0.9.0-py3-none-any.whl.metadata (34 kB)
  Using cached pymupdf-1.25.5-cp39-abi3-win_amd64.whl.metadata (3.4 kB)
Using cached tabulate-0.9.0-py3-none-any.whl (35 kB)
Using cached pymupdf-1.25.5-cp39-abi3-win_amd64.whl (16.6 MB)
Note: you may need to restart the kernel to use updated packages.


In [7]:
import fitz  
import pandas as pd
import re
from io import StringIO

In [9]:
import fitz  # PyMuPDF
import pandas as pd
from io import StringIO
import os

# Delete old nodes.csv if it exists
if os.path.exists("nodes.csv"):
    os.remove("nodes.csv")

# Load PDF and extract text
doc = fitz.open("Project_Provided_Data.pdf")
text = "".join(page.get_text() for page in doc)

# Extract the Neighborhoods and Districts section
start = "Neighborhoods and Districts"
end = "Important Facilities"
block = text.split(start)[1].split(end)[0].strip()

# Clean and prepare lines
lines = [line.strip() for line in block.splitlines() if line.strip() and not line.startswith("CSE112")]

# Parse into DataFrame
df = pd.read_csv(StringIO("\n".join(lines)), header=None)
df.columns = ['ID', 'Name', 'Population', 'Type', 'X', 'Y']

# Save to CSV
df.to_csv("nodes.csv", index=False)
print(f"✅ nodes.csv created with {len(df)} rows.")


✅ nodes.csv created with 18 rows.


In [6]:
if os.path.exists("facilities.csv"):
    os.remove("facilities.csv")

# Load PDF and extract text
doc = fitz.open("Project_Provided_Data.pdf")
text = "".join(page.get_text() for page in doc)

# Extract the Important Facilities section
start = "Important Facilities"
end = "Road Network Data"
block = text.split(start)[1].split(end)[0].strip()

# Clean and prepare lines
lines = [line.strip() for line in block.splitlines() if line.strip() and not line.startswith("CSE112")]

# Parse into DataFrame
df = pd.read_csv(StringIO("\n".join(lines)), header=None)
df.columns = ['ID', 'Name', 'Type', 'X', 'Y']

# Save to CSV
df.to_csv("facilities.csv", index=False)
print(f"✅ facilities.csv created with {len(df)} rows.")


✅ facilities.csv created with 12 rows.


In [11]:
import fitz  # PyMuPDF
import pandas as pd
from io import StringIO
import os

# Delete old file if it exists
if os.path.exists("roads_existing.csv"):
    os.remove("roads_existing.csv")

# Load and extract text
doc = fitz.open("Project_Provided_Data.pdf")
text = "".join(page.get_text() for page in doc)

# Extract block
start = "Existing Roads"
end = "Potential New Roads"
block = text.split(start)[1].split(end)[0].strip()

# Clean and collect valid lines (must have exactly 5 comma-separated values)
lines = []
for line in block.splitlines():
    line = line.strip()
    if not line or line.startswith("CSE112"):
        continue
    parts = [x.strip() for x in line.split(',')]
    if len(parts) == 5:
        lines.append(",".join(parts))

# Parse into DataFrame
df = pd.read_csv(StringIO("\n".join(lines)), header=None)
df.columns = ['FromID', 'ToID', 'Distance', 'Capacity', 'Condition']

# Save to CSV
df.to_csv("roads_existing.csv", index=False)
print(f"✅ roads_existing.csv created with {len(df)} rows.")


✅ roads_existing.csv created with 28 rows.


In [12]:
if os.path.exists("roads_potential.csv"):
    os.remove("roads_potential.csv")

# Load and extract text
doc = fitz.open("Project_Provided_Data.pdf")
text = "".join(page.get_text() for page in doc)

# Extract Potential New Roads section
start = "Potential New Roads"
end = "Traffic Flow Data"
block = text.split(start)[1].split(end)[0].strip()

# Clean and collect valid lines (must have exactly 5 fields)
lines = []
for line in block.splitlines():
    line = line.strip()
    if not line or line.startswith("CSE112"):
        continue
    parts = [x.strip() for x in line.split(',')]
    if len(parts) == 5:
        lines.append(",".join(parts))

# Parse into DataFrame
df = pd.read_csv(StringIO("\n".join(lines)), header=None)
df.columns = ['FromID', 'ToID', 'Distance', 'Capacity', 'Cost']

# Save to CSV
df.to_csv("roads_potential.csv", index=False)
print(f"✅ roads_potential.csv created with {len(df)} rows.")

✅ roads_potential.csv created with 15 rows.


In [18]:
import fitz  # PyMuPDF
import pandas as pd
from io import StringIO
import os

# Delete old file if it exists
if os.path.exists("traffic_flows.csv"):
    os.remove("traffic_flows.csv")

# Load and extract text
doc = fitz.open("Project_Provided_Data.pdf")
text = "".join(page.get_text() for page in doc)

# Extract Traffic Flow Patterns section
start = "Traffic Flow Patterns"
end = "Public Transportation Data"
block = text.split(start)[1].split(end)[0].strip()

# Clean and collect valid lines (must have exactly 5 fields)
lines = []
for line in block.splitlines():
    line = line.strip()
    if not line or line.startswith("CSE112"):
        continue
    parts = [x.strip() for x in line.split(',')]
    if len(parts) == 5:
        # Replace non-standard dash (en dash –) with hyphen (-)
        parts[0] = parts[0].replace('–', '-').replace('—', '-')
        lines.append(",".join(parts))

# Parse into DataFrame
df = pd.read_csv(StringIO("\n".join(lines)), header=None)
df.columns = ['RoadID', 'Morning', 'Afternoon', 'Evening', 'Night']

# Save to CSV
df.to_csv("traffic_flows.csv", index=False)
print(f"✅ traffic_flows.csv created with {len(df)} rows and clean RoadID format.")


✅ traffic_flows.csv created with 28 rows and clean RoadID format.


In [14]:
# Delete old file if it exists
if os.path.exists("metro_lines.csv"):
    os.remove("metro_lines.csv")

# Load and extract text
doc = fitz.open("Project_Provided_Data.pdf")
text = "".join(page.get_text() for page in doc)

# Extract Current Metro Lines section
start = "Current Metro Lines"
end = "Current Bus Routes"
block = text.split(start)[1].split(end)[0].strip()

# Clean and prepare lines
lines = []
for line in block.splitlines():
    line = line.strip()
    if not line or line.startswith("CSE112"):
        continue
    parts = [x.strip() for x in line.split(',', 3)]  # Split into max 4 parts
    if len(parts) == 4:
        # Quote the stations field if it contains commas
        if ',' in parts[2] and not parts[2].startswith('"'):
            parts[2] = f'"{parts[2]}"'
        lines.append(",".join(parts))

# Parse into DataFrame
df = pd.read_csv(StringIO("\n".join(lines)), header=None)
df.columns = ['LineID', 'Name', 'Stations', 'DailyPassengers']

# Save to CSV
df.to_csv("metro_lines.csv", index=False)
print(f"✅ metro_lines.csv created with {len(df)} rows.")


✅ metro_lines.csv created with 4 rows.


In [15]:
# Delete old file if it exists
if os.path.exists("bus_routes.csv"):
    os.remove("bus_routes.csv")

# Load and extract text from PDF
doc = fitz.open("Project_Provided_Data.pdf")
text = "".join(page.get_text() for page in doc)

# Extract the "Current Bus Routes" section
start = "Current Bus Routes"
end = "Public Transportation Demand"
block = text.split(start)[1].split(end)[0].strip()

# Clean and prepare lines
lines = []
for line in block.splitlines():
    line = line.strip()
    if not line or line.startswith("CSE112"):
        continue
    parts = [x.strip() for x in line.split(',', 3)]  # split into 4 fields
    if len(parts) == 4:
        # Quote the Stops field if it contains commas
        if ',' in parts[1] and not parts[1].startswith('"'):
            parts[1] = f'"{parts[1]}"'
        lines.append(",".join(parts))

# Parse into DataFrame
df = pd.read_csv(StringIO("\n".join(lines)), header=None)
df.columns = ['RouteID', 'Stops', 'BusesAssigned', 'DailyPassengers']

# Save to CSV
df.to_csv("bus_routes.csv", index=False)
print(f"✅ bus_routes.csv created with {len(df)} rows.")

✅ bus_routes.csv created with 10 rows.


In [16]:
# Delete old file if it exists
if os.path.exists("pt_demand.csv"):
    os.remove("pt_demand.csv")

# Load and extract text
doc = fitz.open("Project_Provided_Data.pdf")
text = "".join(page.get_text() for page in doc)

# Extract the "Public Transportation Demand" section
start = "Public Transportation Demand"
block = text.split(start)[1].strip()

# Clean and prepare lines
lines = []
for line in block.splitlines():
    line = line.strip()
    if not line or line.startswith("CSE112"):
        continue
    parts = [x.strip() for x in line.split(',')]
    if len(parts) == 3:
        lines.append(",".join(parts))

# Parse into DataFrame
df = pd.read_csv(StringIO("\n".join(lines)), header=None)
df.columns = ['FromID', 'ToID', 'DailyPassengers']

# Save to CSV
df.to_csv("pt_demand.csv", index=False)
print(f"✅ pt_demand.csv created with {len(df)} rows.")

✅ pt_demand.csv created with 18 rows.
